In [ ]:
# 安裝必要庫
!pip install emcee

# 導入庫
import numpy as np
import emcee
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from google.colab import files

"""
Dependencies:
- numpy==1.23.5
- emcee==3.1.4
- matplotlib==3.7.1
- pandas==2.0.3
- scipy==1.10.1
Data: SPARC dataset (Lelli et al. 2016), available at http://astroweb.cwru.edu/SPARC/
"""

# 上傳數據並讀取
uploaded = files.upload()
column_names = ['galaxy_id', 'rad[kpc]', 'vobs[km/s]', 'errv[km/s]', 'mass[M_sun]']
sparc_data = pd.read_csv(list(uploaded.keys())[0], sep='\s+', names=column_names, skiprows=0)
galaxy_ids = sparc_data['galaxy_id'].unique()

# 基本常數與單位轉換
G = 6.674e-11         # 萬有引力常數 (m^3 kg^-1 s^-2)
M_sun = 1.989e30      # 太陽質量 (kg)
kpc_to_m = 3.085e19   # 1 kpc 轉為 m
a0 = 1.2e-10          # MOND 加速度常數 (m/s^2)

# 定義 MCMC 參數維度，用於後續統計
ndim_q = 2
ndim_L = 4
ndim_M = 3
nwalkers = 32

# ------------------------------------------------------------------------
# 1) 現象學版四元數模型 (Section 3.2: ε(r) = 2 · tanh(r/rs))
#    自由參數：M_solar, rs_kpc
# ------------------------------------------------------------------------
def rotation_curve_quat_pheno(r_kpc, M_solar, rs_kpc):
    r_m  = r_kpc * kpc_to_m
    rs_m = rs_kpc * kpc_to_m
    v_baryon = np.sqrt(G * M_solar * M_sun * (1 - np.exp(-r_m/rs_m)) / r_m)
    eps = 2.0 * np.tanh(r_m / rs_m)
    v2 = v_baryon**2 + eps * (r_m/rs_m)
    return np.sqrt(v2) * 1e-3  # km/s

# ------------------------------------------------------------------------
# 2) ΛCDM 模型
# ------------------------------------------------------------------------
def rotation_curve_LCDM(r_kpc, M_solar, rs_kpc, M_dm, rs_dm):
    r_m = r_kpc * kpc_to_m
    v_baryon = np.sqrt(G * M_solar * M_sun / r_m)
    v_dm = np.sqrt(G * M_dm * M_sun * (1 - np.exp(-r_m/(rs_dm*kpc_to_m))) / r_m)
    return np.sqrt(v_baryon**2 + v_dm**2) * 1e-3  # km/s

# ------------------------------------------------------------------------
# 3) MOND 模型（含外部場效應）
# ------------------------------------------------------------------------
def rotation_curve_MOND_ext(r_kpc, M_solar, rs_kpc, g_ext):
    r_m = r_kpc * kpc_to_m
    v_N = np.sqrt(G * M_solar * M_sun / r_m)
    a_N = v_N**2 / r_m
    mu = a_N / (a0 + g_ext)
    v_MOND = v_N * np.sqrt(0.5 + 0.5 * np.sqrt(1 + 4/mu**2))
    return v_MOND * 1e-3  # km/s

# ------------------------------------------------------------------------
# 共用：計算置信區間
# ------------------------------------------------------------------------
def get_confidence_intervals(samples, percentiles=[16,84]):
    return [np.percentile(samples[:, i], percentiles) for i in range(samples.shape[1])]

# ------------------------------------------------------------------------
# MCMC: 1) Quaternionic 現象學版 (2 參數)
# ------------------------------------------------------------------------
def log_likelihood_quat(theta, r, v, v_err):
    M_solar, rs_kpc = theta
    v_model = rotation_curve_quat_pheno(r, M_solar, rs_kpc)
    return -0.5 * np.sum(((v - v_model)/v_err)**2)

def log_prior_quat(theta):
    M_solar, rs_kpc = theta
    if 1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20:
        return 0.0
    return -np.inf

def log_probability_quat(theta, r, v, v_err):
    lp = log_prior_quat(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_quat(theta, r, v, v_err)

# ------------------------------------------------------------------------
# MCMC: 2) ΛCDM (4 參數)
# ------------------------------------------------------------------------
def log_likelihood_LCDM(theta, r, v, v_err):
    M_solar, rs_kpc, M_dm, rs_dm = theta
    v_model = rotation_curve_LCDM(r, M_solar, rs_kpc, M_dm, rs_dm)
    return -0.5 * np.sum(((v - v_model)/v_err)**2)

def log_prior_LCDM(theta):
    M_solar, rs_kpc, M_dm, rs_dm = theta
    if (1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20
        and 1e8 < M_dm    < 1e12 and 1   < rs_dm  < 50):
        return 0.0
    return -np.inf

def log_probability_LCDM(theta, r, v, v_err):
    lp = log_prior_LCDM(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_LCDM(theta, r, v, v_err)

# ------------------------------------------------------------------------
# MCMC: 3) MOND (3 參數)
# ------------------------------------------------------------------------
def log_likelihood_MOND(theta, r, v, v_err):
    M_solar, rs_kpc, g_ext = theta
    v_model = rotation_curve_MOND_ext(r, M_solar, rs_kpc, g_ext)
    return -0.5 * np.sum(((v - v_model)/v_err)**2)

def log_prior_MOND(theta):
    M_solar, rs_kpc, g_ext = theta
    if 1e8 < M_solar < 1e11 and 0.1 < rs_kpc < 20 and 0 < g_ext < 1e-9:
        return 0.0
    return -np.inf

def log_probability_MOND(theta, r, v, v_err):
    lp = log_prior_MOND(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + log_likelihood_MOND(theta, r, v, v_err)

# ------------------------------------------------------------------------
# 逐星系擬合與結果彙整
# ------------------------------------------------------------------------
results = []
for galaxy in galaxy_ids:
    data = sparc_data[sparc_data['galaxy_id']==galaxy].dropna()
    data = data[(data['vobs[km/s]']>0)&(data['errv[km/s]']>0)]
    if len(data) < 4:
        continue

    r_kpc = data['rad[kpc]'].values
    v_obs = data['vobs[km/s]'].values
    v_err = data['errv[km/s]'].values.clip(min=0.1)
    M0     = data['mass[M_sun]'].iloc[0]

    # --- Quaternionic 現象學版 MCMC ---
    p0_q = np.array([M0, 8.5]) * (1 + 0.1*np.random.randn(nwalkers, ndim_q))
    sampler_q = emcee.EnsembleSampler(nwalkers, ndim_q, log_probability_quat,
                                      args=(r_kpc, v_obs, v_err))
    sampler_q.run_mcmc(p0_q, 2000, progress=True)
    flat_q = sampler_q.get_chain(discard=500, flat=True)
    M_fit_q, rs_fit_q = np.median(flat_q, axis=0)
    ci_q = get_confidence_intervals(flat_q)
    v_fit_q = rotation_curve_quat_pheno(r_kpc, M_fit_q, rs_fit_q)
    chi2_q = np.sum(((v_obs - v_fit_q)/v_err)**2)
    dof_q  = len(v_obs) - ndim_q
    redchi2_q = chi2_q/dof_q if dof_q>0 else np.nan
    AIC_q = chi2_q + 2*ndim_q
    BIC_q = chi2_q + ndim_q*np.log(len(v_obs))

    # --- ΛCDM MCMC ---
    p0_L = np.array([M0, 3.0, M0*5, 15.0]) * (1 + 0.1*np.random.randn(nwalkers, ndim_L))
    sampler_L = emcee.EnsembleSampler(nwalkers, ndim_L, log_probability_LCDM,
                                      args=(r_kpc, v_obs, v_err))
    sampler_L.run_mcmc(p0_L, 2000, progress=True)
    flat_L = sampler_L.get_chain(discard=500, flat=True)
    M_fit_L, rs_fit_L, Mdm_fit, rsdm_fit = np.median(flat_L, axis=0)
    v_fit_L = rotation_curve_LCDM(r_kpc, M_fit_L, rs_fit_L, Mdm_fit, rsdm_fit)
    chi2_L = np.sum(((v_obs - v_fit_L)/v_err)**2)
    dof_L  = len(v_obs) - ndim_L
    redchi2_L = chi2_L/dof_L if dof_L>0 else np.nan
    AIC_L = chi2_L + 2*ndim_L
    BIC_L = chi2_L + ndim_L*np.log(len(v_obs))

    # --- MOND MCMC ---
    p0_M = np.array([M0, 3.0, 1e-10]) * (1 + 0.1*np.random.randn(nwalkers, ndim_M))
    sampler_M = emcee.EnsembleSampler(nwalkers, ndim_M, log_probability_MOND,
                                      args=(r_kpc, v_obs, v_err))
    sampler_M.run_mcmc(p0_M, 2000, progress=True)
    flat_M = sampler_M.get_chain(discard=500, flat=True)
    M_fit_M, rs_fit_M, gext_fit = np.median(flat_M, axis=0)
    v_fit_M = rotation_curve_MOND_ext(r_kpc, M_fit_M, rs_fit_M, gext_fit)
    chi2_M = np.sum(((v_obs - v_fit_M)/v_err)**2)
    dof_M  = len(v_obs) - ndim_M
    redchi2_M = chi2_M/dof_M if dof_M>0 else np.nan
    AIC_M = chi2_M + 2*ndim_M
    BIC_M = chi2_M + ndim_M*np.log(len(v_obs))

    # 擇優 (AIC 最小)
    aic_map = {'Quat': AIC_q, 'LCDM': AIC_L, 'MOND': AIC_M}
    best = min(aic_map, key=aic_map.get)

    results.append({
        'Galaxy': galaxy,
        # chi2 & dof
        'chi2_q': chi2_q, 'dof_q': dof_q, 'redchi2_q': redchi2_q, 'AIC_q': AIC_q, 'BIC_q': BIC_q,
        'chi2_L': chi2_L, 'dof_L': dof_L, 'redchi2_L': redchi2_L, 'AIC_L': AIC_L, 'BIC_L': BIC_L,
        'chi2_M': chi2_M, 'dof_M': dof_M, 'redchi2_M': redchi2_M, 'AIC_M': AIC_M, 'BIC_M': BIC_M,
        'Best': best
    })

    # 繪圖
    plt.figure(figsize=(8,5))
    plt.errorbar(r_kpc, v_obs, yerr=v_err, fmt='o', alpha=0.5, label='Data')
    plt.plot(r_kpc, v_fit_q, label='Quat-Phenomenology')
    plt.plot(r_kpc, v_fit_L, label='ΛCDM')
    plt.plot(r_kpc, v_fit_M, label='MOND')
    plt.xscale('log')
    plt.xlabel('Radius [kpc]')
    plt.ylabel('v_rot [km/s]')
    plt.title(f'Rotation Curve Fits — {galaxy}')
    plt.legend(); plt.grid(which='both', ls='--')
    plt.show()

# 匯出並顯示結果
df = pd.DataFrame(results)
print(df[['Galaxy','redchi2_q','AIC_q','redchi2_L','AIC_L','redchi2_M','AIC_M','Best']])
df.to_csv("sparc_model_comparison.csv", index=False)
print("Results saved to sparc_model_comparison.csv")

# ------------------------------------------------------------------------
# 最後統計累計
# ------------------------------------------------------------------------
total_chi2_q = df['chi2_q'].sum()
total_dof_q = df['dof_q'].sum()
total_redchi2_q = total_chi2_q / total_dof_q

total_chi2_L = df['chi2_L'].sum()
total_dof_L = df['dof_L'].sum()
total_redchi2_L = total_chi2_L / total_dof_L

total_chi2_M = df['chi2_M'].sum()
total_dof_M = df['dof_M'].sum()
total_redchi2_M = total_chi2_M / total_dof_M

print(f"\nTotal reduced chi2 (Quaternionic): {total_redchi2_q:.2f}")
print(f"Total reduced chi2 (LCDM):       {total_redchi2_L:.2f}")
print(f"Total reduced chi2 (MOND):       {total_redchi2_M:.2f}")

best_model_counts = df['Best'].value_counts()
print("\n最佳模型統計：")
print(best_model_counts)

outliers = df[df['chi2_q'] > 10000]
if not outliers.empty:
    print("\nOutliers with chi2_q > 10000:")
    print(outliers[['Galaxy', 'chi2_q', 'AIC_q']].to_string(index=False))
else:
    print("\nNo outliers with chi2_q > 10000.")